In [16]:
from kfp import dsl, compiler, components

In [ ]:
hf_downloader_component_url = "_components/hf_downloader.yaml"
download_artifact = components.load_component_from_file(hf_downloader_component_url)

In [ ]:
@dsl.pipeline
def dowload_hf_artifact(
    repo_id: str,
    repo_type: str = "model",
    filename: str = "model.pt",
):
    """Download a model from Hugging Face Hub.

    Args:
        :param repo_id: The Hugging Face Hub repo ID of the model to download.
    """
    download_document_step = download_artifact(
        repo_id=repo_id, repo_type=repo_type, filename=filename)
    download_document_step.set_caching_options(False)

In [ ]:
compiler.Compiler().compile(dowload_hf_artifact, package_path='download_hf_artifact.yaml')